In [ ]:
from smolagents import CodeAgent, InferenceClientModel, Tool, DuckDuckGoSearchTool
from dotenv import load_dotenv
import os
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from langfuse import get_client
from huggingface_hub import notebook_login
import time
import functools
notebook_login()
load_dotenv()
angfuse_key = os.getenv('ANGFUSE_SECRET_KEY')
model_id = "Qwen/Qwen2.5-14B-Instruct"

In [ ]:
def robust_llm_call(func):
    """
    A standard Python decorator that retries the decorated function 
    if the LLM server disconnects or returns a 503/404/Connection error.
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        max_retries = 5
        base_wait_time = 2  # Start with 2 seconds
        
        for attempt in range(max_retries):
            try:
                # Attempt to execute the decorated method (e.g., forward)
                return func(*args, **kwargs)
                
            except Exception as e:
                error_msg = str(e)
                
                # Check for common server-side transient errors
                # We also catch "404" specifically because sometimes routers momentarily lose the model
                is_transient = (
                    "503" in error_msg or 
                    "Service Temporarily Unavailable" in error_msg or 
                    "Connection error" in error_msg or
                    "404" in error_msg  # Sometimes helpful for momentary router glitches
                )
                
                if is_transient:
                    wait_time = base_wait_time * (2 ** attempt) # Exponential backoff: 2, 4, 8...
                    print(f"\n[System] Connection dropped in '{func.__name__}'. Retrying in {wait_time}s... (Attempt {attempt+1}/{max_retries})")
                    time.sleep(wait_time)
                else:
                    # If it's a real code error (e.g., TypeError), crash immediately
                    raise e
                    
        raise Exception(f"Max retries ({max_retries}) reached. The server is persistently unavailable.")
        
    return wrapper

In [ ]:
class RecipeRetrieverTool(Tool):
    name = "retrieve_recipe"
    description = "Retrieves the best matching recipe/s from the database. Returns raw title and ingredients."
    inputs = {
        "query": {"type": "string", "description": "Dish name or ingredients."},
    }
    
    output_type = "string"
    
    def __init__(self, k: int = 1, **kwargs):
        """Initialize the recipe retrieval tool.
        
        Args:
            k: Number of top recipes to retrieve (default: 1)
        """
        super().__init__(**kwargs)
        self.k = k
        
        print(f"Loading recipe retrieval system with top {k} results")
        
        # Load metadata from embeddings file
        self.metadata = []
        with open('recipes_for_embeddings.jsonl', 'r') as f:
            for line in f:
                self.metadata.append(json.loads(line))
        
        # Load full recipe details (with ingredients and directions)
        with open('full_format_recipes.json', 'r') as f:
            full_recipes = json.load(f)
        
        # HashTable for optimized lookup
        self.recipe_lookup = {r.get('title', '').strip(): r for r in full_recipes if r.get('title')}
        
        # Load embedding model
        print("Loading embedding model BAAI/bge-m3...")
        self.embed_model = SentenceTransformer('BAAI/bge-m3')
        
        # Load FAISS index
        print("Loading FAISS index...")
        self.index = faiss.read_index('recipe_index.faiss')
        
        print(f"✓ Recipe retrieval system loaded: {len(self.metadata)} recipes indexed")
    @robust_llm_call # Decorator to retry on connection errors
    def forward(self, query: str) -> str:
        """Search for recipes matching the query.
        
        Args:
            query: Natural language search query
            
        Returns:
            Formatted string with recipe titles, ingredients, and directions
        """
        
        try:
            if not query or not isinstance(query, str):
                return "Found 0 recipes."
            # 1. Embed the Query and Ensure Float 32 (Required by FAISS)
            query_vec = self.embed_model.encode([query], convert_to_tensor=False)
            query_vec = np.array(query_vec).astype('float32')
            
            # 2. Normalize euclidean distance
            faiss.normalize_L2(query_vec)
            
            # 3. Search
            _, indices = self.index.search(query_vec, self.k)
            
            # 4. Retrieve
            retrieved_docs = [self.metadata[idx] for idx in indices[0] if idx != -1]
            
            # 5. Format Output
            output = f"Found {len(retrieved_docs)} recipes matching '{query}':\n\n"
            
            for i, doc in enumerate(retrieved_docs, 1):
                title = doc['title'].strip()
                full_recipe = self.recipe_lookup.get(title)
                
                output += f"{'='*40} Recipe {i} {'='*40}\n"
                output += f"TITLE: {title}\n"
                
                if full_recipe:
                    output += "INGREDIENTS:\n" + "\n".join([f" - {ing}" for ing in full_recipe.get('ingredients', [])])
                    output += "\n\nDIRECTIONS:\n" + "\n".join([f" {j}. {step}" for j, step in enumerate(full_recipe.get('directions', []), 1)])
                else:
                    # Fallback
                    output += f"SUMMARY: {doc.get('text_for_embedding', 'No details available')}"
                
                output += "\n\n"
                
            return output
        except Exception as e:
            return f"Found 0 recipes. Error during retrieval: {str(e)}"

In [ ]:
class RecipeAdapterTool(Tool):
    name = "adapt_recipe"
    description = "Rewrites a recipe to comply with a specific dietary constraint (e.g., 'Make this vegan')."
    inputs = {
        "recipe_text": {"type": "string", "description": "The original recipe text."},
        "target_diet": {"type": "string", "description": "The target diet (e.g., 'vegan', 'gluten-free')."}
    }
    output_type = "string"

    def __init__(self, model_engine, **kwargs):
        super().__init__(**kwargs)
        self.model_engine = model_engine
    @robust_llm_call # Decorator to retry on connection errors
    def forward(self, recipe_text: str, target_diet: str) -> str:
        # We construct a prompt for the LLM to do the rewriting
        prompt = f"""
        You are an expert chef. Rewrite the following recipe to be strictly {target_diet}.
        
        Rules:
        1. Replace ONLY forbidden ingredients with best culinary substitutes.
        2. Keep the original formatting.
        3. Do not change the dish identity (e.g., 'Beef Stew' becomes 'Lentil Stew', not 'Salad').
        
        Original Recipe:
        {recipe_text}
        
        Rewritten Recipe:
        """
        
        # Call the LLM (using smolagents' model wrapper)
        messages = [{"role": "user", "content": prompt}]
        
        # fixed to return content instead of whole message for validation
        response = self.model_engine(messages)
        if hasattr(response, "content"):
            return response.content
        else:
            return str(response)
        

In [ ]:
class RecipeValidatorTool(Tool):
    name = "validate_recipe"
    description = "Checks recipe compliance. Returns strictly 'PASS' or 'FAIL'."
    inputs = {
        "recipe_text": {"type": "string", "description": "The recipe text."},
        "constraint": {"type": "string", "description": "The diet (e.g., 'vegan')."}
    }
    output_type = "string"

    def __init__(self, model_engine, **kwargs):
        super().__init__(**kwargs)
        self.model_engine = model_engine

    @robust_llm_call
    def forward(self, recipe_text: str, constraint: str) -> str:
        prompt = f"""
        Review this recipe for the strict constraint: "{constraint}".
        RECIPE: {recipe_text[:3000]} # Truncate to avoid context errors
        
        If ANY forbidden ingredient is present, output FAIL.
        If it is safe, output PASS.
        
        Final Answer (Strictly 'PASS' or 'FAIL'):
        """
        messages = [{"role": "user", "content": prompt}]
        
        try:
            response = self.model_engine(messages)
            content = response.content if hasattr(response, "content") else str(response)
            
            # Deterministic Parsing
            if "FAIL" in content.upper():
                return "FAIL"
            return "PASS"
        except Exception:
            # Fallback to FAIL on model error to be safe
            return "FAIL"

In [ ]:
from ddgs import DDGS

class WebSearchTool(Tool):
    name = "duckduckgo_search"
    description = "Searches the web for recipes. Returns the content of the best result as a string."
    inputs = {
        "query": {"type": "string", "description": "Search query."}
    }
    output_type = "string"

    def forward(self, query: str) -> str:
        try:
            results = DDGS().text(query, max_results=3)
            if not results:
                return "No recipes found on the web."
            
            # Format the list of results into a single string for the Agent
            formatted_results = "Web Search Results:\n"
            for res in results:
                formatted_results += f"TITLE: {res['title']}\nCONTENT: {res['body']}\nURL: {res['href']}\n\n"
            
            return formatted_results
        except Exception as e:
            return f"Web search failed: {str(e)}"

## Instantiate the model

In [ ]:
model = InferenceClientModel(model_id=model_id)

# Instantiate the tools

In [ ]:
retriever_tool = RecipeRetrieverTool()
adapter_tool = RecipeAdapterTool(model_engine=model)
validator_tool = RecipeValidatorTool(model_engine=model)
search_tool = WebSearchTool()

# Define prompt for the multiagent

In [151]:
SYSTEM_PROMPT = """
You are an intelligent Culinary Agent powered by Qwen-72B.
Your goal is to find, adapt, and validate recipes programmatically.

AVAILABLE TOOLS:
1. `retrieve_recipe(query)`: Searches internal DB. Returns a formatted STRING.
2. `adapt_recipe(recipe_text, target_diet)`: Rewrites the recipe text. Returns a STRING.
3. `validate_recipe(recipe_text, constraint)`: Checks compliance. Returns strictly "PASS" or "FAIL".
4. `duckduckgo_search(query)`: Web search fallback. Returns a STRING.

### CRITICAL CODING RULES (VIOLATION = CRASH)
1. **NO BACKSLASHES (`\\`)**:
   - Use parentheses `()` for long function calls.

2. **NO PASSIVE PRINTING**:
   - **RIGHT:** `if validate_recipe(...) == "FAIL":`
   - **REASON:** Handle logical flows in code.

3. **VARIABLE SAFETY**:
   - **STATELESS EXECUTION**: Your variables do NOT persist between turns.
   - **ALWAYS** initialize `recipe_candidate = None` at the start of your code.
   - **NEVER** copy-paste massive recipe text unless absolutely necessary (keeps code clean).

### STANDARD OPERATING PROCEDURE (SOP):

1. **SETUP**:
   - Initialize: `recipe_candidate = None`
   - Define `search_query` based on the user's request.
   - Define `target_diet` (e.g., "vegan", "keto").

2. **ACQUISITION**:
   - Call `recipe_candidate = retrieve_recipe(search_query)`.
   
   - **Check for Failure**:
       # If retriever returns empty/short text or "Found 0", fallback to Web
       if not recipe_candidate or len(recipe_candidate) < 100 or "Found 0" in recipe_candidate:
           recipe_candidate = duckduckgo_search(f"{search_query} recipe {target_diet}")

3. **VALIDATION & REPAIR**:
   - Call `status = validate_recipe(recipe_candidate, constraint=target_diet)`.
   
   - **IF** `status == "FAIL"`:
       # Adapt the recipe
       recipe_candidate = adapt_recipe(recipe_text=recipe_candidate, target_diet=target_diet)
       final_answer(recipe_candidate)
   
   - **ELSE**:
       final_answer(recipe_candidate)

4. **FINAL SUBMISSION**:
   - Ensure `final_answer` is executed in all paths.

OUTPUT FORMAT:
- Strict Python code block.
"""

# Instantiate the multiagent

In [152]:
agent = CodeAgent(
    tools=[
        retriever_tool, 
        adapter_tool, 
        validator_tool, 
        search_tool
    ],
    model=model,
    add_base_tools=True, # This enables the "scratchpad" where Qwen writes its Python logic
    max_steps=12, # Give it enough turns to think -> adapt -> validate
    )


In [153]:
def run_agent_safe(agent, prompt):
    """
    Retries the AGENT'S BRAIN (Planning/Reasoning) if the server fails.
    """
    max_retries = 5
    for attempt in range(max_retries):
        try:
            return agent.run(prompt)
        except Exception as e:
            error_msg = str(e)
            if "503" in error_msg or "Service Temporarily Unavailable" in error_msg or "404" in error_msg:
                wait_time = 2 ** (attempt + 1)
                print(f"\n🧠 Agent 'Brain' glitch (Attempt {attempt+1}/{max_retries}). Retrying decision in {wait_time}s...")
                time.sleep(wait_time)
            else:
                raise e # Real error, let it crash
    raise Exception("Agent Brain failed. Server is down.")

In [ ]:
# response = run_agent("I need a high protein recipe that includes chicken.")
# print(response)

In [154]:

@robust_llm_call # handle random sever disconnects
def start_session():
    # 1 get the initial query from the user
    
    
    current_recipe_context = None
    if os.path.exists("last_recipe_state.txt"):
        print("found previous session! Loading last recipe...")
        with open("last_recipe_state.txt", "r") as f:
            current_recipe_context = f.read()
            print(f"RESUMED CONTEXT: {current_recipe_context[:100]}...")

    user_request = input("\nEnter your food request (or 'clear' to start fresh): ")
    
    if user_request.lower() == 'clear':
        current_recipe_context = None
        if os.path.exists("last_recipe_state.txt"): os.remove("last_recipe_state.txt")
        user_request = input("Enter new food request: ")


    while True:
        # (Construct prompt as before...)
        if not current_recipe_context:
            full_prompt = f"{SYSTEM_PROMPT}\nUSER REQUEST: {user_request}"
        else:
            full_prompt = f"""
            {SYSTEM_PROMPT}
            CONTEXT: Previous recipe provided.
            PREVIOUS RECIPE: {current_recipe_context}
            USER FEEDBACK: {user_request}
            TASK: Adapt PREVIOUS RECIPE to USER FEEDBACK. Validate it.
            """
        
        print("\n--- Agent is thinking... ---\n")

        try:
            # Run the agent
            response = run_agent_safe(agent, full_prompt)
            
            # --- SAVE STATE IMMEDIATELY FOR HANDLING SERVER DISCONNECT ---
            current_recipe_context = response
            with open("last_recipe_state.txt", "w") as f:
                f.write(str(response))
            # ------------------------------

            print(f"\n[AGENT]:\n{response}\n")
            
        except Exception as e:
            print(f"CRITICAL ERROR: {e}")
            print("The agent state has been preserved. You can try running the cell again.")
            break 
        
        print("-" * 50)
        user_request = input("Feedback (or type 'exit'): ")
        if user_request.lower() in ['exit', 'quit', 'q']:
            break

In [155]:
start_session()


--- Agent is thinking... ---



╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an intelligent Culinary Agent powered by Qwen-72B.                                                      │
│ Your goal is to find, adapt, and validate recipes programmatically.                                             │
│                                                                                                                 │
│ AVAILABLE TOOLS:                                                                                                │
│ 1. `retrieve_recipe(query)`: Searches internal DB. Returns a formatted STRING.                                  │
│ 2. `adapt_recipe(recipe_text, target_diet)`: Rewrites the recipe text. Returns a STRING.                        │
│ 3. `validate_recipe(recipe_text, constraint)`: Checks compliance. Returns strictly "PASS" or "FAIL".            │
│ 4. `duckduckgo_search(query)`: Web search fallback. Returns a STRING.                                           │
│                                                                                                                 │
│ ### CRITICAL CODING RULES (VIOLATION = CRASH)                                                                   │
│ 1. **NO BACKSLASHES (`\`)**:                                                                                    │
│    - Use parentheses `()` for long function calls.                                                              │
│                                                                                                                 │
│ 2. **NO PASSIVE PRINTING**:                                                                                     │
│    - **RIGHT:** `if validate_recipe(...) == "FAIL":`                                                            │
│    - **REASON:** Handle logical flows in code.                                                                  │
│                                                                                                                 │
│ 3. **VARIABLE SAFETY**:                                                                                         │
│    - **STATELESS EXECUTION**: Your variables do NOT persist between turns.                                      │
│    - **ALWAYS** initialize `recipe_candidate = None` at the start of your code.                                 │
│    - **NEVER** copy-paste massive recipe text unless absolutely necessary (keeps code clean).                   │
│                                                                                                                 │
│ ### STANDARD OPERATING PROCEDURE (SOP):                                                                         │
│                                                                                                                 │
│ 1. **SETUP**:                                                                                                   │
│    - Initialize: `recipe_candidate = None`                                                                      │
│    - Define `search_query` based on the user's request.                                                         │
│    - Define `target_diet` (e.g., "vegan", "keto").                                                              │
│                                                                                                                 │
│ 2. **ACQUISITION**:                                                                                             │
│    - Call `recipe_candidate = retrieve_recipe(search_query)`.                                                   │
│                                                                                                                 │
│    - **Check for Failure**:                           

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  recipe_candidate = None                                                                                          
  search_query = "vegan lasagna"                                                                                   
  target_diet = "vegan"                                                                                            
                                                                                                                   
  # Attempt to retrieve the recipe from the internal database                                                      
  recipe_candidate = retrieve_recipe(search_query)                                                                 
                                                                                                                   
  # Check if retrieval was successful                                                                              
  if not recipe_candidate or len(recipe_candidate) < 100 or "Found 0" in recipe_candidate:                         
      # Fallback to web search if retrieval failed                                                                 
      recipe_candidate = duckduckgo_search(f"{search_query} recipe {target_diet}")                                 
                                                                                                                   
  # Validate the recipe for vegan compliance                                                                       
  status = validate_recipe(recipe_candidate, constraint=target_diet)                                               
                                                                                                                   
  # Adapt the recipe if it doesn't comply with the vegan diet                                                      
  if status == "FAIL":                                                                                             
      recipe_candidate = adapt_recipe(recipe_text=recipe_candidate, target_diet=target_diet)                       
      final_answer(recipe_candidate)                                                                               
  else:                                                                                                            
      final_answer(recipe_candidate)                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: **Rewritten Recipe:**

======================================== Recipe 1 ========================================  
TITLE: Roasted Vegetable Lasagna  
INGREDIENTS:  
- 1 lb plum tomatoes, cut in 1/4-inch slices  
- 1 lb zucchini, cut in 1/4-inch slices  
- 1 lb yellow squash, cut in 1/4-inch slices  
- 2 red bell peppers, cut in 1-inch strips  
- 2 green bell peppers, cut in 1-inch strips  
- 1/2 lb mushroom caps, cut in 1/4-inch slices  
- 1 tsp salt  
- 1 tbsp olive oil  
- Vegetable-oil cooking spray  
- 1 flax egg (1 tablespoon ground flaxseed mixed with 3 tablespoons water)  
- 2 containers (15 oz each) "lite" ricotta (ensure it's vegan)  
- 2 tbsp bottled pesto sauce (ensure it's vegan)  
- 1/3 cup nutritional yeast (substitute for Parmesan)  
- 3 cans (14 1/2 oz each) diced tomatoes with garlic and onion  
- 12 oven-ready lasagna noodles (ensure they're vegan)  
- 2 cups vegan shredded mozzarella cheese (ensure it melts well)

DIRECTIONS:  
1. Preheat oven to 475°F. Toss plum tomatoes, zucchini, squash, peppers, mushrooms, salt, and oil in a bowl. Coat a
baking sheet with cooking spray and place vegetables on it; roast 30 minutes. In a bowl, mix flax egg, ricotta, 
pesto, and nutritional yeast. Coat a 9" x 12" baking pan with cooking spray. Spread 1 can tomatoes on the bottom. 
Top with 3 noodles. Spoon 1 1/4 cups ricotta mix over noodles, then a layer of 3 cups vegetables, then 1/2 cup 
vegan mozzarella. Repeat this layer, starting with tomatoes. Add final can tomatoes, three noodles, remaining 
ricotta mix and vegetables. Top lasagna with last 3 noodles and 1 cup vegan mozzarella. Cover with foil. Bake 30 
minutes.

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  recipe_text = """                                                                                                
  ======================================== Recipe 1 ========================================                       
  TITLE: Roasted Vegetable Lasagna                                                                                 
  INGREDIENTS:                                                                                                     
  - 1 lb plum tomatoes, cut in 1/4-inch slices                                                                     
  - 1 lb zucchini, cut in 1/4-inch slices                                                                          
  - 1 lb yellow squash, cut in 1/4-inch slices                                                                     
  - 2 red bell peppers, cut in 1-inch strips                                                                       
  - 2 green bell peppers, cut in 1-inch strips                                                                     
  - 1/2 lb mushroom caps, cut in 1/4-inch slices                                                                   
  - 1 tsp salt                                                                                                     
  - 1 tbsp olive oil                                                                                               
  - Vegetable-oil cooking spray                                                                                    
  - 1 flax egg (1 tablespoon ground flaxseed mixed with 3 tablespoons water)                                       
  - 2 containers (15 oz each) "lite" ricotta (ensure it's vegan)                                                   
  - 2 tbsp bottled pesto sauce (ensure it's vegan)                                                                 
  - 1/3 cup nutritional yeast (substitute for Parmesan)                                                            
  - 3 cans (14 1/2 oz each) diced tomatoes with garlic and onion                                                   
  - 12 oven-ready lasagna noodles (ensure they're vegan)                                                           
  - 2 cups vegan shredded mozzarella cheese (ensure it melts well)                                                 
                                                                                                                   
  DIRECTIONS:                                                                                                      
  1. Preheat oven to 475°F. Toss plum tomatoes, zucchini, squash, peppers, mushrooms, salt, and oil in a bowl.     
  Coat a baking sheet with cooking spray and place vegetables on it; roast 30 minutes. In a bowl, mix flax egg,    
  ricotta, pesto, and nutritional yeast. Coat a 9" x 12" baking pan with cooking spray. Spread 1 can tomatoes on   
  the bottom. Top with 3 noodles. Spoon 1 1/4 cups ricotta mix over noodles, then a layer of 3 cups vegetables,    
  then 1/2 cup vegan mozzarella. Repeat this layer, starting with tomatoes. Add final can tomatoes, three          
  noodles, remaining ricotta mix and vegetables. Top lasagna with last 3 noodles and 1 cup vegan mozzarella.       
  Cover with foil. Bake 30 minutes.                                                                                
  """                                                                                                              
                                                                                                                   
  # Replace nutritional yeast with vegan cheese                                                                    
  recipe_text = recipe_text.replace("1/3 cup nutritional yeast (substitute for Parmesan)", "1/3 cup vegan          
  Parmesan cheese")                                      

Final answer: 
======================================== Recipe 1 ========================================
TITLE: Roasted Vegetable Lasagna
INGREDIENTS:
- 1 lb plum tomatoes, cut in 1/4-inch slices
- 1 lb zucchini, cut in 1/4-inch slices
- 1 lb yellow squash, cut in 1/4-inch slices
- 2 red bell peppers, cut in 1-inch strips
- 2 green bell peppers, cut in 1-inch strips
- 1/2 lb mushroom caps, cut in 1/4-inch slices
- 1 tsp salt
- 1 tbsp olive oil
- Vegetable-oil cooking spray
- 1 flax egg (1 tablespoon ground flaxseed mixed with 3 tablespoons water)
- 2 containers (15 oz each) "lite" ricotta (ensure it's vegan)
- 2 tbsp bottled pesto sauce (ensure it's vegan)
- 1/3 cup vegan Parmesan cheese
- 3 cans (14 1/2 oz each) diced tomatoes with garlic and onion
- 12 oven-ready lasagna noodles (ensure they're vegan)
- 2 cups vegan shredded mozzarella cheese (ensure it melts well)

DIRECTIONS:
1. Preheat oven to 475°F. Toss plum tomatoes, zucchini, squash, peppers, mushrooms, salt, and oil in a bowl. Coat a
baking sheet with cooking spray and place vegetables on it; roast 30 minutes. In a bowl, mix flax egg, ricotta, 
pesto, and nutritional yeast. Coat a 9" x 12" baking pan with cooking spray. Spread 1 can tomatoes on the bottom. 
Top with 3 noodles. Spoon 1 1/4 cups ricotta mix over noodles, then a layer of 3 cups vegetables, then 1/2 cup 
vegan mozzarella. Repeat this layer, starting with tomatoes. Add final can tomatoes, three noodles, remaining 
ricotta mix and vegetables. Top lasagna with last 3 noodles and 1 cup vegan mozzarella. Cover with foil. Bake 30 
minutes.

[Step 1: Duration 33.34 seconds| Input tokens: 3,328 | Output tokens: 606]


[AGENT]:

======================================== Recipe 1 ========================================
TITLE: Roasted Vegetable Lasagna
INGREDIENTS:
- 1 lb plum tomatoes, cut in 1/4-inch slices
- 1 lb zucchini, cut in 1/4-inch slices
- 1 lb yellow squash, cut in 1/4-inch slices
- 2 red bell peppers, cut in 1-inch strips
- 2 green bell peppers, cut in 1-inch strips
- 1/2 lb mushroom caps, cut in 1/4-inch slices
- 1 tsp salt
- 1 tbsp olive oil
- Vegetable-oil cooking spray
- 1 flax egg (1 tablespoon ground flaxseed mixed with 3 tablespoons water)
- 2 containers (15 oz each) "lite" ricotta (ensure it's vegan)
- 2 tbsp bottled pesto sauce (ensure it's vegan)
- 1/3 cup vegan Parmesan cheese
- 3 cans (14 1/2 oz each) diced tomatoes with garlic and onion
- 12 oven-ready lasagna noodles (ensure they're vegan)
- 2 cups vegan shredded mozzarella cheese (ensure it melts well)

DIRECTIONS:
1. Preheat oven to 475°F. Toss plum tomatoes, zucchini, squash, peppers, mushrooms, salt, and oil in a bowl. C